In [11]:
import time
import numpy as np
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt

import tdwf

# ===================== Parametri utente =====================
R_ohm = 10_000          # <-- Valore del resistore noto (Ohm)
amp_V = 0.2             # Ampiezza sinusoidale (Vpp/2? NO: qui è picco a picco?)  # vedi nota sotto
f_min = 10              # Hz  (inizio sweep)
f_max = 50_000          # Hz  (fine sweep: alza/abbassa a seconda del C atteso)
n_freq = 30             # numero punti sweep (log-spaced)
cycles_per_record = 20  # n. di periodi da acquisire per stima RMS robusta
settle_time = 0.05      # s da attendere dopo cambio freq prima di campionare
# Nota AD2/tdwf: w1.ampl in genere è Vpp (picco-picco). Se è così nel tuo tdwf,
# lascia amp_V piccolo (0.2–0.5 Vpp) per non saturare e per ridurre non idealità.

# ===================== Inizializzazione AD2 =================
ad2 = tdwf.AD2()
ad2.vdd = 3
ad2.vss = -3
ad2.power(True)

wgen = tdwf.WaveGen(ad2.hdwf)
wgen.w1.ampl = amp_V
wgen.w1.freq = f_min
wgen.w1.offs = 0
wgen.w1.func = tdwf.funcSine
wgen.w1.start()

scope = tdwf.Scope(ad2.hdwf)
scope.npt = 8192
scope.ch1.rng = 2      # range in Volt/div circa: metti 2 o 5 per 0.2–1 Vpp
scope.ch2.rng = 2
scope.trig(True, level=0.02, hist=0.05, sour=tdwf.trigsrcCh1)

# ===================== Utility di misura ====================
def set_scope_fs_for_freq(f_hz, npt=8192, cycles=20, fs_min=10_000, fs_max=1_000_000):
    """
    Sceglie una fs per acquisire 'cycles' periodi in 'npt' campioni:
    fs = npt * f / cycles. Poi clamp tra fs_min e fs_max.
    """
    fs = npt * f_hz / cycles
    fs = max(fs_min, min(fs, fs_max))
    scope.fs = fs

def acquire_rms(f_hz):
    """
    Campiona i due canali e ritorna (Vrms_in, Vrms_c, ratio=Vc/Vin).
    Assumiamo segnali centrati (offs=0). Usiamo RMS diretto.
    """
    set_scope_fs_for_freq(f_hz, npt=scope.npt, cycles=cycles_per_record)
    # piccolo delay per riempire i buffer con la nuova fs
    time.sleep(0.01)
    scope.sample()  # blocking read
    ch1 = np.asarray(scope.ch1.vals)
    ch2 = np.asarray(scope.ch2.vals)
    # rimozione di eventuale DC residua per robustezza
    ch1 = ch1 - np.mean(ch1)
    ch2 = ch2 - np.mean(ch2)
    rms1 = np.sqrt(np.mean(ch1**2))
    rms2 = np.sqrt(np.mean(ch2**2))
    ratio = rms2 / rms1 if rms1 > 0 else np.nan
    return rms1, rms2, ratio

# ===================== Sweep in frequenza ===================
freqs = np.logspace(np.log10(f_min), np.log10(f_max), n_freq)
ratios = np.zeros_like(freqs)
vrms_in = np.zeros_like(freqs)
vrms_c  = np.zeros_like(freqs)

print("== Misura in corso ==")
for i, f in enumerate(freqs):
    wgen.w1.freq = float(f)
    time.sleep(settle_time)
    r1, r2, k = acquire_rms(f)
    vrms_in[i], vrms_c[i], ratios[i] = r1, r2, k
    print(f"f = {f:9.1f} Hz | Vin_rms = {r1:.4f} V | Vc_rms = {r2:.4f} V | |H| = {k:.4f}")

# ===================== Ricerca fc (|H| = 1/sqrt(2)) =========
target = 1/np.sqrt(2)

# cerchiamo il primo punto in cui il modulo scende sotto 0.707
idx = np.where(ratios <= target)[0]
if len(idx) == 0 or idx[0] == 0:
    # non trovato o bordo: fallback semplice (stima grezza)
    # prende il punto più vicino a 0.707
    i_best = np.argmin(np.abs(ratios - target))
    f_c = freqs[i_best]
    note_fc = "stima discreta (nessuna interpolazione possibile)"
else:
    j = idx[0]
    i = j - 1
    # interpolazione lineare di |H| rispetto a log10(f)
    x1, y1 = np.log10(freqs[i]), ratios[i]
    x2, y2 = np.log10(freqs[j]), ratios[j]
    if y2 == y1:
        f_c = 10**x1
        note_fc = "interpolazione degenerata"
    else:
        t = (target - y1) / (y2 - y1)
        x = x1 + t * (x2 - x1)
        f_c = 10**x
        note_fc = "interpolazione su asse log-f"

# ===================== Calcolo C ============================
C = 1 / (2*np.pi*R_ohm*f_c)

# Stima incertezze “minime” (solo discretizzazione + toll. R)
# - incertezza su fc ~ metà distanza log tra i due punti usati per l’interpolazione
if len(idx) > 0 and idx[0] > 0:
    j = idx[0]; i = j-1
    f_lo, f_hi = freqs[i], freqs[j]
    # mezzo intervallo in termini relativi
    df_rel = (f_hi/f_lo)**0.5 - 1
else:
    # se non abbiamo due punti attorno, usa spaziatura media dello sweep
    if n_freq > 1:
        r = (f_max/f_min)**(1/(n_freq-1))
        df_rel = (r**0.5 - 1)
    else:
        df_rel = 0.1
# ipotizza R al 1% (cambia qui se hai 0.1% o 5%)
dR_rel = 0.01

dC_over_C = np.sqrt(df_rel**2 + dR_rel**2)
dC = C * dC_over_C

print("\n== RISULTATI ==")
print(f"R = {R_ohm} Ω (assunta tolleranza ~{dR_rel*100:.1f}%)")
print(f"f_c = {f_c:.3f} Hz  [{note_fc}]")
print(f"C = {C:.6e} F  ±{dC:.6e} F  (≈ {C*1e9:.1f} nF ± {dC*1e9:.1f} nF)")

# ===================== Plot ================================
fig, ax = plt.subplots(figsize=(8,5))
ax.semilogx(freqs, ratios, '-o', label='|Vc/Vin| misurato')
ax.axhline(target, ls='--', lw=1, label='-3 dB (0.707)')
ax.axvline(f_c, ls='--', lw=1, label=f'f_c ≈ {f_c:.1f} Hz')
ax.set_xlabel('Frequenza [Hz]')
ax.set_ylabel('Modulo |H|')
ax.grid(True, which='both', alpha=0.3)
ax.legend()
ax.set_title('Passa-basso RC misurato con AD2')
plt.tight_layout()
plt.show()

# ===================== Cleanup =============================
wgen.w1.stop()
ad2.close()


Dispositivo #1 [SN:210321B5D136, hdwf=1] connesso!
Configurazione #1
== Misura in corso ==
f =      10.0 Hz | Vin_rms = 0.1404 V | Vc_rms = 0.1389 V | |H| = 0.9892
f =      13.4 Hz | Vin_rms = 0.1418 V | Vc_rms = 0.1403 V | |H| = 0.9889
f =      18.0 Hz | Vin_rms = 0.1425 V | Vc_rms = 0.1408 V | |H| = 0.9884
f =      24.1 Hz | Vin_rms = 0.1423 V | Vc_rms = 0.1405 V | |H| = 0.9873
f =      32.4 Hz | Vin_rms = 0.1417 V | Vc_rms = 0.1397 V | |H| = 0.9852
f =      43.4 Hz | Vin_rms = 0.1418 V | Vc_rms = 0.1391 V | |H| = 0.9815
f =      58.3 Hz | Vin_rms = 0.1417 V | Vc_rms = 0.1383 V | |H| = 0.9756
f =      78.1 Hz | Vin_rms = 0.1417 V | Vc_rms = 0.1367 V | |H| = 0.9643
f =     104.8 Hz | Vin_rms = 0.1417 V | Vc_rms = 0.1340 V | |H| = 0.9455
f =     140.6 Hz | Vin_rms = 0.1417 V | Vc_rms = 0.1296 V | |H| = 0.9149
f =     188.6 Hz | Vin_rms = 0.1417 V | Vc_rms = 0.1226 V | |H| = 0.8653
f =     253.0 Hz | Vin_rms = 0.1417 V | Vc_rms = 0.1126 V | |H| = 0.7942
f =     339.3 Hz | Vin_rms = 0.14